In [2]:
import os
import pandas as pd
import numpy as np

In [ ]:
def read_tcpflow(path):
    #first : read part 0
    num_of_file = 10
    f = open(path+"part-00000.csv","r")
    tcpflow = pd.read_csv(f)
    f.close()
    
    #second: read othed files one by one
    for i in range(1, num_of_file):
        f = open(path+"part-0000"+str(i)+".csv", "r")
        temp_df = pd.read_csv(f)
        f.close()
        tcpflow=tcpflow.append(temp_df)
        print(i)
        
    
    return tcpflow
    
tcpflow = read_tcpflow("F://A_sophomore_s2//dataFoutain//复赛数据//dataset_2//tcpflow//csv//")

1
2
3
4


In [ ]:
tcpflow.head(10)

In [ ]:
tcpflow.shape #make sure you read all data

In [ ]:
#convert attribute recore_time into timestamp type
tcpflow['record_time'] = pd.to_datetime(tcpflow['record_time'])

In [ ]:
def get_time(tcpflow):
    s1 = tcpflow["record_time"].dt.month.astype(str) + "-"+ tcpflow["record_time"].dt.day.astype(str)
    return s1.unique()


time_list = get_time(tcpflow)

In [ ]:
print(time_list)

In [ ]:
def get_protocol_type(tcpflow, protocol_file):
    
    
    protocol_group = tcpflow.groupby(["protocol"])
    protocol_dict = protocol_group.groups
    protocol_list= protocol_dict.keys()
    print(len(protocol_list))  # get number of protocol type
    
    
    protocol_csv = open(protocol_file ,"w")
    protocol_csv.write("protocol,number\n")
    

    
    for key  in protocol_group.groups.keys() :

        value = protocol_group.groups[key]
        protocol_csv.write(key+","+str(len(value))+"\n")


    protocol_csv.close()
    return 

protocol_file = "F://A_sophomore_s2//dataFoutain//复赛汇总//协议数量统计.csv"
get_protocol_type(tcpflow, protocol_file)

In [7]:
def get_sum_flow(tcpflow,flow_file_path ):
    
    #groupby(source ip ), groupby(source ip )
    #每一个ip: 
    #up = src_ip.up + des_ip. down
    #down = src_ip.down+des_ip.up
    
    
    src_group = tcpflow.groupby("source_ip")
    des_group = tcpflow.groupby("destination_ip")
    
    src_ip_set = set(src_group.groups.keys()) 
    des_ip_set = set(des_group.groups.keys() ) 
    
    up_flow_dict = { }
    down_flow_dict = { }
    
    for src_ip, src_df in src_group:
        up_as_src = src_df["uplink_length"].sum()
        down_as_src = src_df["downlink_length"].sum()
        #print("1: ",up_as_src,down_as_src)
        if(src_ip in des_ip_set):
            des_df = tcpflow[tcpflow['destination_ip'].isin([src_ip])] #find all row whose ip attribute is "src_ip", the result is a df
            up_as_des = des_df['downlink_length'].sum()
            down_as_des = des_df["uplink_length"].sum()
        else:
            up_as_des = 0
            down_as_des = 0
        #print("1: ",up_as_des,down_as_des)
        up_flow = up_as_src + up_as_des
        down_flow = down_as_src + down_as_des
        up_flow_dict[src_ip] = up_flow
        down_flow_dict[src_ip] = down_flow
        
    print("next")
    for des_ip, des_df in des_group:
        if(des_ip in src_ip_set):
            continue
        else:

            up_as_des = des_df['downlink_length'].sum()
            down_as_des = des_df["uplink_length"].sum()
        
            up_flow = up_as_des
            down_flow = down_as_des
            up_flow_dict[des_ip] = up_flow
            down_flow_dict[des_ip] = down_flow
    
        
    flow_file = open(flow_file_path+"IP流量统计.csv", "w")
    flow_file.write("ip,up_length,down_length,sum_length\n")
    
    for key in up_flow_dict.keys():
        print(key)
        sum_flow_str = str(up_flow_dict[key]+down_flow_dict[key] )
        flow_file.write(key+","+str(up_flow_dict[key])+","+str(down_flow_dict[key])+","+sum_flow_str+"\n" )
            
        
    
    
    
    return 

# flow_file_path = "F://A_sophomore_s2//dataFoutain//复赛汇总//"  #填写你想要存储的路径  
# get_sum_flow(tcpflow，flow_file_path)

In [81]:
def get_ip_pair_sum_flow(tcpflow,path):
    
    #以ip pair, protocol 作为主码
    #对每一个group 写ip pair protocol 的 df 流量加和
    file = open(path+"IP对分协议流量统计.csv","w")
    file.write("source_ip,destination_ip,protocol,uplink_length,downlink_length,sum_length\n")
    
    ip_pair_protocol_group = tcpflow.groupby(["source_ip", "destination_ip","protocol"]) 
    for ipp , ipp_df in ip_pair_protocol_group:
        up_flow = ipp_df["uplink_length"].sum()
        down_flow = ipp_df["downlink_length"].sum()
        sum_flow_str = str(up_flow+ down_flow)
        file.write(ipp[0]+","+ipp[1]+","+ipp[2]+","+str(up_flow)+","+str(down_flow)+","+sum_flow_str+"\n")
        
path = "F://A_sophomore_s2//dataFoutain//复赛汇总//"  #将路径改为你需要存储汇总文件的路径 
get_ip_pair_sum_flow(tcpflow, path)

In [18]:
def get_first_second_ifference(first_file, second_file):
    #得到初赛和复赛的表格，并且将两个表做外连接
    
    #初赛数据格式： ip,down_length,up_length,sum_length
    #复赛数据格式：ip,up_length,down_length,sum_length
    
   
    f1 = open(first_file)
    f2 = open(second_file)
    
    first = pd.read_csv(f1)
    second = pd.read_csv(f2)
    
    f1.close()
    f2.close()
    #将ip作为连接列
    new = pd.merge(second, first, how = "outer",left_on='ip', right_on='ip')
    new = new.fillna(0)#只在其中某一个表出现的ip， 连接后会被设置为NaN， 为了方便分析， 填充为0
    
    new.to_csv("F://A_sophomore_s2//dataFoutain//复赛汇总//初赛复赛IP流量对比.csv")
    
    return

first_file = "F://A_sophomore_s2//dataFoutain//复赛汇总//初赛_分IP真实流量统计.csv"
second_file = "F://A_sophomore_s2//dataFoutain//复赛汇总//复赛_IP流量统计.csv"
get_first_second_ifference(first_file, second_file)



In [33]:
def difference_analysis(flow_file):
    #传入流量对比文件名， 分析初赛和复赛的ip流量变化 
    #分析流量增大的ip， 单独列出一个csv
    #分析流量减少的ip， 单独列出一个csv
    #flow的格式为：ip,s_up_length,s_down_length,s_sum_length,f_down_length,f_up_length,f_sum_length
    #s代表复赛， f代表初赛

    f= open(flow_file)
    flow = pd.read_csv(f)
    f.close()
    
    #首先分析上行流量变大的ip
    #对两个异常IP 单独处理, 全部分析完成后， 再找出这一列， 然后进行
    
    
    #down_df = pd.DataFrame(columns = ["ip","s_down_length", "f_down_length","down_increment"])
    up_df = pd.DataFrame(columns = ["ip","s_up_length", "f_up_length","up_increment"])
    

    #分析下行流量变大的ip
    flow["down_increment"] = flow["s_down_length"]-flow["f_down_length"]
    down_df = flow[["ip","s_down_length", "f_down_length","down_increment"]][flow['down_increment'] > 0]
    down_df.to_csv("F://A_sophomore_s2//dataFoutain//复赛汇总//下行流量变大ip.csv")
    
    
    
    
    #分析上行流量变大的ip
    flow["up_increment"] = flow["s_up_length"] - flow["f_up_length"]
    up_df = flow[["ip","s_up_length", "f_up_length","up_increment"]][flow['up_increment'] > 0]
    up_df.to_csv("F://A_sophomore_s2//dataFoutain//复赛汇总//上行流量变大ip.csv")
    
    #分析总流量变大
    flow["sum_increment"] = flow["s_sum_length"] - flow["f_sum_length"]
    sum_df = flow[["ip","s_sum_length", "f_sum_length","sum_increment"]][flow['sum_increment'] > 0]
    sum_df.to_csv("F://A_sophomore_s2//dataFoutain//复赛汇总//总流量变大ip.csv")
    
    
    

    return
    
flow_file = "F://A_sophomore_s2//dataFoutain//复赛汇总//初赛复赛IP流量对比.csv"
difference_analysis(flow_file)

In [19]:
    if (not os.path.exists(flow_file_path) ):
            os.mkdir(flow_file_path)  
        get_sum_flow(group,flow_file_path )

        print(name, "finished")
    return

In [4]:
def get_common_ip(flow_file):
    #给出初赛、复赛中都有的ip
    common_df = flow[["ip","s_down_length", "f_down_length","down_increment"]][flow['down_increment'] > 0]
    
    
    
    
    return

flow_file = "F://A_sophomore_s2//dataFoutain//复赛汇总//初赛复赛IP流量对比.csv"
get_common_ip(flow_file)

In [ ]:
def get_tcpflow_with_timestamp(tcpflow):
    tcpflow_stamp = tcpflow
    tcpflow_stamp["day_time"] = "_"
    tcpflow_stamp["day_time"] = tcpflow_stamp["record_time"].dt.month.astype(str)+tcpflow_stamp["day_time"] + tcpflow_stamp["record_time"].dt.day.astype(str)+tcpflow_stamp["day_time"]+tcpflow_stamp["record_time"].dt.hour.astype(str)
    
    return tcpflow_stamp
tcpflow_stamp = get_tcpflow_with_timestamp(tcpflow)

In [22]:
tcpflow_stamp

,Unnamed: 0,destination_ip,destination_port,downlink_length,protocol,record_time,source_ip,uplink_length,day_time
0,0,118.194.56.52,443,4729,ssl,2019-05-30 19:58:58.035,10.56.131.163,2164,5_30_19
1,1,202.75.220.12,5226,373,unknown,2019-05-30 22:27:23.798,10.54.251.114,412,5_30_22
2,2,183.129.205.92,5227,397,unknown,2019-05-30 22:27:23.905,10.54.251.114,425,5_30_22
3,3,110.76.20.18,443,547,unknown,2019-05-30 22:28:41.793,10.54.251.114,445,5_30_22
4,4,10.24.64.39,6682,125,redis,2019-05-30 23:42:45.509,10.49.137.136,3501,5_30_23
5,5,10.24.64.39,6682,118,redis,2019-05-30 23:42:45.536,10.49.137.136,807,5_30_23
6,6,10.24.64.39,6682,118,redis,2019-05-30 23:42:45.545,10.49.137.136,879,5_30_23
7,7,10.24.64.39,6682,118,redis,2019-05-30 23:42:45.554,10.49.137.136,1077,5_30_23
8,8,10.24.64.39,6682,118,redis,2019-05-30 23:42:45.554,10.49.137.136,879,5_30_23
9,9,10.24.64.39,6682,202,redis,2019-05-30 23:43:45.278,10.49.137.136,3035,5_30_23


In [ ]:
def  generate_time_set():
    day_time_list = ['5_30','5_31','6_1','6_2','6_3','6_4','6_5','6_6','6_7','6_8','6_9','6_10','6_11','6_12','6_13']
    hour_set = set()
    
    for i in range (0, len(day_time_list)):
        for j in range(0, 24):
            hour_set.add(day_time_list[i]+"_"+str(j))
    
    return hour_set


In [72]:
def get_IP_flow_by_hour(tcpflow_stamp, ip):
    #任何一个有时间戳的df 都可以统计
    #找到
    #然后
    
    
    full_hour_set = generate_time_set()
    #先验知识： 没有src ip ==  des ip的行
    #所以含有这个ip的行可以分des含有ip的行和src含有此ip的行两种情况讨论
    des_df = tcpflow_stamp[tcpflow_stamp['destination_ip'].isin([ip])] #find all row whose ip attribute is "src_ip", the result is a df

    src_df = tcpflow_stamp[tcpflow_stamp['source_ip'].isin([ip])]
    
    #groupby 之后计算src 的接收和发送 的流量， 然后存成dict
    #des df 里面再分， 算出接收和发送， 加上去
    #最后检测set， 不在里面的就设置为0 
    
    receive_flow = {}
    send_flow =  {}
    
    hour_set = set()
    
    des_group = des_df.groupby("day_time")
    src_group = src_df.groupby("day_time")
    
    print("1")
    for name, group in des_group : #des up = "downlink_length"  down = "uplink_length"
        up_flow = group["downlink_length"].sum()
        down_flow = group["uplink_length"].sum()
        
        send_flow[name] = up_flow
        receive_flow[name] = down_flow
        
        hour_set.add(name)
    
    print("2")
    
    for name, group in src_group:
        up_flow = group["uplink_length"].sum()
        down_flow = group["downlink_length"].sum()
        
        if name in hour_set :
            send_flow[name] =send_flow[name]+ up_flow
            receive_flow[name] = receive_flow[name]+down_flow
        else:
            send_flow[name] = up_flow
            receive_flow[name] = down_flow
            hour_set.add(name)
    
    #对于从来没有出现过的时段， 设置为0
    
    non_exist_time = full_hour_set - hour_set
    print(non_exist_time)
    
    
    print("3")
    for time in non_exist_time :
        send_flow[time] = 0
        receive_flow[time] = 0
    
    #将dict写入, 为了保证有序， 先排序
    key_list = list( receive_flow.keys() )
    key_list.sort()
    file = "F://A_sophomore_s2//dataFoutain//复赛汇总//ip_时间段_汇总//"+ip+".csv"
    f = open(file, "w",encoding="utf-8")
    f.write("day_time,send_flow,receive_flow,sum_flow\n")
    for i in range(0, len(key_list)):
        key = key_list[i]
        send = send_flow[key]
        receive = receive_flow[key]
        sum_str = str(send+receive)
        
        f.write(key+","+str(send)+","+str(receive)+","+sum_str+"\n")
        
    f.close()
    
    
    
    
    return

In [ ]:
def get_time_protocol(tcpflow_stamp, ip ):
    #目的是给出某一个时间段
    #
    full_hour_set = generate_time_set()
    #先验知识： 没有src ip ==  des ip的行
    #所以含有这个ip的行可以分des含有ip的行和src含有此ip的行两种情况讨论
    des_df = tcpflow_stamp[tcpflow_stamp['destination_ip'].isin([ip])] #find all row whose ip attribute is "src_ip", the result is a df

    src_df = tcpflow_stamp[tcpflow_stamp['source_ip'].isin([ip])]
    
    #groupby 之后计算src 的接收和发送 的流量， 然后存成dict
    #des df 里面再分， 算出接收和发送， 加上去
    '''
    得到每一个时间段的group之后，对这个group按照协议groupby, 有一个dict， 然后每一个df计计算sum， 向dict里面计算sum，
    if这个协议已经有了， 那么直接加和， if没有， 那么直接等于
    
    最后那个没有的时间段， 直接等于空字符串
    '''
    
    #最后检测set， 不在里面的就设置为0 
    
    
    
    receive_flow = {}
    send_flow =  {}
    
    time_dict = { } #每一个时间段对应的protocol , protocol也是一个dict, 里面存储着： 协议:流量: m时间段：{协议:}
    
    hour_set = set()
    
    des_group = des_df.groupby("day_time")
    src_group = src_df.groupby("day_time")
    
    
    
    print("1")
    for name, group in des_group : #des up = "downlink_length"  down = "uplink_length"
        protocol_dict = {}
        
        up_flow = group["downlink_length"].sum()
        down_flow = group["uplink_length"].sum()
        
        send_flow[name] = up_flow
        receive_flow[name] = down_flow
        
        hour_set.add(name)
        protocol_dict = get_protocol_flow(group,2)
        time_dict[name] = protocol_dict 
#         protocol_group = 
    
    
    
    print("2")
    
    for name, group in src_group:
        up_flow = group["uplink_length"].sum()
        down_flow = group["downlink_length"].sum()
        protocol_dict = get_protocol_flow(group,1)
        
        if name in hour_set :
            send_flow[name] =send_flow[name]+ up_flow
            receive_flow[name] = receive_flow[name]+down_flow
            #在里面， 那么说明原先有， 所以情况比较复杂
            #首先判断这次的protocol有没有重复
            #protocol在里面，直接send receive相加
            #protocol不在里面， 直接加入立刻
            protocol_src = protocol_dict.keys()
            protocol_des_set = set( time_dict[name].keys() )
            for i in range(0,len( protocol_src) ):
                if protocol_src[i] in protocol_des_set:
                    time_dict[name][protocol_src[i]]["send"]=time_dict[name]["send"]+protocol_dict["send"]
                    time_dict[name][protocol_src[i]]["receive"]=time_dict[name]["receive"]+protocol_dict["receive"]
                else:
                    time_dict[name][protocol_src[i]] = protocol_dict
            
        else:
            send_flow[name] = up_flow
            receive_flow[name] = down_flow
            hour_set.add(name)
            time_dict[name] =  protocol_dict
            #不在src里面， 比较简单，直接让dict等于他即可
            
        
    
    #对于从来没有出现过的时段， 设置为0
    
    non_exist_time = full_hour_set - hour_set
    print(non_exist_time)
    
    
    print("3")
    for time in non_exist_time :
        send_flow[time] = 0
        receive_flow[time] = 0
        time_dict[time] = ""
    
    
    #将dict写入, 为了保证有序， 先排序
    key_list = list( receive_flow.keys() )
    key_list.sort()
    file = "F://A_sophomore_s2//dataFoutain//复赛汇总//ip_时间段_协议流量统计//"+ip+".csv"
    f = open(file, "w",encoding="utf-8")
    f.write("day_time,send_flow,receive_flow,sum_flow,protocol\n")
    for i in range(0, len(key_list)):
        key = key_list[i]
        send = send_flow[key]
        receive = receive_flow[key]
        sum_str = str(send+receive)
        protocol_str = str(time_dict[key])
        
        protocol_str=protocol_str.replace(",","|")
        f.write(key+","+str(send)+","+str(receive)+","+sum_str+","+protocol_str+"\n")
        
    f.close()
    
    return

In [88]:
def get_protocol_flow(df,df_type ):
    #给一个df, 计算每个protocol的receive 和 send
    #protocol: [uplink, downlink]
    protocol_dict = { }
    
    protocol_group =  df.groupby("protocol")
    
    if(df_type == 1):  #1: des
        for pro, group in protocol_group:
            send_flow = group["downlink_length"].sum()
            receive_flow = group["uplink_length"].sum()
            protocol_dict[pro] = {"send":send_flow, "receive":receive_flow}
            
    elif(df_type == 2): #2:src
        for pro, group in protocol_group:
            receive_flow = group["downlink_length"].sum()
            send_flow = group["uplink_length"].sum()
            protocol_dict[pro] = {"send":send_flow, "receive":receive_flow}

    
    return protocol_dict

In [ ]:
get_time_protocol(tcpflow_stamp, "10.21.12.148")

In [98]:
s = "1,2,3"
s.replace(",","")

'123'